# Introduction to FermiLib 0.1

## $-1.7 \, a^\dagger_3 a_1$ and $(1 + 2i) \, a^\dagger_4 a^\dagger_2 a_3 a_0$ are FermionTerms

In [ ]:
from fermion_operators import FermionTerm

n_qubits = 5
term_1 = FermionTerm(n_qubits, -1.7, [(3, 1), (1, 0)])
term_2 = FermionTerm(n_qubits, 1. + 2.j, [(4, 1), (2, 1), (3, 0), (0, 0)])

print term_1
print term_2

## FermionTerms support many built-in operators

In [ ]:
print 2. * term_1
print term_1 / 2.

In [ ]:
print term_1 == term_2
print term_1 != term_2

In [ ]:
print abs(term_2)
print len(term_2)

In [ ]:
print term_1 ** 2
print term_1 * term_2

In [ ]:
for x in term_2:
    print x

In [ ]:
print (4, 1) in term_2
print (1, 0) in term_2

In [ ]:
print term_1
term_1 *= 3.
print term_1
term_1 /= 3.
print term_1

## FermionOperators store the sum of FermionTerms

In [ ]:
operator_1 = 8.2 * term_1 - 3.0 * term_2
operator_2 = term_2 ** 2 - operator_1
print operator_1
print operator_2

In [ ]:
print operator_1 - 2. * operator_2
print term_1 + operator_2

In [ ]:
print operator_1
operator_1 -= 1000. * term_1 * term_2
print operator_1

In [ ]:
operator_2 **= 2
print operator_2

In [ ]:
print term_1 in operator_1
print term_1 in operator_2

In [ ]:
for term in operator_2:
    print term
    term *= 0.0001
print operator_2

In [ ]:
print operator_2[term_1.operators]
print operator_1[term_1.operators]

## Of course, there are many custom methods

In [ ]:
print term_1.commutator(term_2)
print term_1.commutator(term_1)

In [ ]:
print term_2
print term_2.hermitian_conjugated()

In [ ]:
term_3 = term_2 * term_2.hermitian_conjugated()
print term_3
print term_3.is_normal_ordered()
print term_3.normal_ordered()

In [ ]:
print term_1.is_molecular_term()
print term_2.is_molecular_term()

In [ ]:
print operator_1.list_terms()
print operator_1.list_coefficients()

## We can map to QubitTerms and QubitOperators (and back)

In [ ]:
print term_1.jordan_wigner_transform()
print term_1.bravyi_kitaev_transform()

In [ ]:
qubit_op_1 = term_1.jordan_wigner_transform()
qubit_op_2 = operator_1.jordan_wigner_transform()
print qubit_op_1
print qubit_op_2

In [ ]:
for x in qubit_op_1:
    print x
    print x.reverse_jordan_wigner()

In [ ]:
qubit_op_2 -= 2. * qubit_op_1 ** 2
print qubit_op_2

## QubitTerms and QubitOperators can map to SparseOperators

In [ ]:
sparse_op_1 = qubit_op_1.get_sparse_operator()
print sparse_op_1
sparse_op_1 *= 2.
print
print sparse_op_1

In [ ]:
fermion_term = term_1 + term_1.hermitian_conjugated()
print fermion_term
qubit_term = fermion_term.jordan_wigner_transform()
print qubit_term
sparse_operator = qubit_term.get_sparse_operator()
print sparse_operator.get_eigenspectrum()

In [ ]:
print sparse_operator.is_hermitian()
ground_state_energy, ground_state = sparse_operator.get_ground_state()
print ground_state_energy
print sparse_operator.expectation(ground_state)

## Lattice models

In [ ]:
from hubbard import fermi_hubbard
x_dimension = 3
y_dimension = 2
tunneling = 2.
coulomb = 1.
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 0

hubbard_model = fermi_hubbard(
    x_dimension, y_dimension, tunneling, coulomb,
    chemical_potential, magnetic_field, periodic, spinless)
print hubbard_model

In [ ]:
qubit_hamiltonian = hubbard_model.jordan_wigner_transform()
print qubit_hamiltonian

## Jellium

In [1]:
from jellium import jellium_model

n_dimensions = 1
grid_length = 3
length_scale = 2.
spinless = 1

momentum_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                 spinless, momentum_space=True)
momentum_qubit = momentum_jellium.jordan_wigner_transform()
print momentum_jellium
print momentum_qubit

4.93480220054 [2+ 2]
0.318309886184 [1+ 0+ 1 0]
0.318309886184 [2+ 0+ 2 0]
0.318309886184 [1+ 2+ 1 2]
0.318309886184 [2+ 1+ 2 1]
0.318309886184 [0+ 1+ 0 1]
4.93480220054 [0+ 0]
0.318309886184 [0+ 2+ 0 2]

(-0.159154943092+0j) Z0 Z2
(-2.14909121409+0j) Z2
(-0.159154943092+0j) Z1 Z2
(-2.14909121409+0j) Z0
(-0.159154943092+0j) Z0 Z1
(4.45733737127+0j) I
(0.318309886184+0j) Z1



In [2]:
position_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                 spinless, momentum_space=False)
position_qubit = position_jellium.jordan_wigner_transform()
print position_jellium
print position_qubit

-1.64493406685 [1+ 2]
3.2898681337 [2+ 2]
-1.64493406685 [0+ 1]
-0.318309886184 [1+ 1 0+ 0]
-0.318309886184 [0+ 0 1+ 1]
-0.318309886184 [1+ 1 2+ 2]
-0.318309886184 [2+ 2 1+ 1]
-1.64493406685 [0+ 2]
-1.64493406685 [2+ 1]
-1.64493406685 [1+ 0]
-1.64493406685 [2+ 0]
3.2898681337 [0+ 0]
-0.318309886184 [0+ 0 2+ 2]
3.2898681337 [1+ 1]
-0.318309886184 [2+ 2 0+ 0]

(-0.159154943092+0j) Z0 Z2
(-1.32662418066+0j) Z2
(-0.159154943092+0j) Z1 Z2
(-1.32662418066+0j) Z0
(-0.822467033424+0j) Y0 Y1
(-0.159154943092+0j) Z0 Z1
(-0.822467033424+0j) X0 Z1 X2
(-1.32662418066+0j) Z1
(-0.822467033424+0j) Y1 Y2
(-0.822467033424+0j) X1 X2
(-0.822467033424+0j) Y0 Z1 Y2
(4.45733737127+0j) I
(-0.822467033424+0j) X0 X1



In [3]:
momentum_sparse = momentum_qubit.get_sparse_operator()
position_sparse = position_qubit.get_sparse_operator()
print momentum_sparse.get_eigenspectrum() - position_sparse.get_eigenspectrum()

[ -2.77555756e-17  -1.38777878e-16   2.66453526e-15  -8.88178420e-16
   8.88178420e-16  -8.88178420e-16   0.00000000e+00   1.77635684e-15]


In [7]:
n_dimensions = 1
length_scale = 2.
spinless = 1

for grid_length in range(3, 17):
    
    if not grid_length % 2:
        continue
    
    momentum_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                     spinless, momentum_space=True)
    momentum_qubit = momentum_jellium.jordan_wigner_transform()
    
    position_jellium = jellium_model(n_dimensions, grid_length, length_scale,
                                     spinless, momentum_space=False)
    position_qubit = position_jellium.jordan_wigner_transform()
    
    print 'At {} qubits...'.format(momentum_qubit.n_qubits)
    print 'Momentum space Hamiltonian has {} terms.'.format(len(momentum_qubit))
    print 'Position space Hamiltonian has {} terms.\n'.format(len(position_qubit))

At 3 qubits...
Momentum space Hamiltonian has 7 terms.
Position space Hamiltonian has 13 terms.

At 5 qubits...
Momentum space Hamiltonian has 56 terms.
Position space Hamiltonian has 36 terms.

At 7 qubits...
Momentum space Hamiltonian has 197 terms.
Position space Hamiltonian has 71 terms.

At 9 qubits...
Momentum space Hamiltonian has 478 terms.
Position space Hamiltonian has 118 terms.

At 11 qubits...
Momentum space Hamiltonian has 947 terms.
Position space Hamiltonian has 177 terms.

At 13 qubits...
Momentum space Hamiltonian has 1652 terms.
Position space Hamiltonian has 248 terms.

At 15 qubits...
Momentum space Hamiltonian has 2641 terms.
Position space Hamiltonian has 331 terms.



## MolecularData class stores data about molecules

In [8]:
from molecular_data import MolecularData, _PERIODIC_TABLE

# Set parameters to make a simple molecule.
diatomic_bond_length = 8.
geometry = [('Na', (0., 0., 0.)), ('Cl', (0., 0., diatomic_bond_length))]
basis = '6-31g'
multiplicity = 1
charge = 0
autosave = False
description = str(diatomic_bond_length)

# Make molecule and print out a few interesting facts about it.
molecule = MolecularData(geometry, basis, multiplicity, charge, description, autosave)
print('Molecule has automatically generated name {}'.format(molecule.name))
print('Information about this molecule would be saved at:\n{}\n'.format(molecule.data_handle()))
print('This molecule has {} atoms and {} electrons.'.format(
    molecule.n_atoms, molecule.n_electrons))
for atom, atomic_number in zip(molecule.atoms, molecule.protons):
    print('Contains {} atom, which has {} protons.'.format(atom, atomic_number))

Molecule has automatically generated name Na1-Cl1_6-31g_singlet_8.0
Information about this molecule would be saved at:
/usr/local/google/home/babbush/Desktop/fermilib/src/data/Na1-Cl1_6-31g_singlet_8.0

This molecule has 2 atoms and 28 electrons.
Contains Na atom, which has 11 protons.
Contains Cl atom, which has 17 protons.


In [9]:
from chemical_series import make_atomic_ring, make_atom

# Now let's use chemical_series.py to make a hydrogen ring.
atom_spacing = 0.7414
basis = 'sto-3g'
atom_type = 'H'
biggest_ring = 20
print('\nAbout to make some hydrogen rings.')
for n_atoms in range(4, 20, 2):
    hydrogen_ring = make_atomic_ring(
        n_atoms, atom_spacing, basis, atom_type, autosave=autosave)
    print('Molecule has automatically generated name {}'.format(hydrogen_ring.name))
    print('This molecule has {} atoms and {} electrons.'.format(
        hydrogen_ring.n_atoms, hydrogen_ring.n_electrons))


About to make some hydrogen rings.
Molecule has automatically generated name H4_sto-3g_singlet_ring_0.7414
This molecule has 4 atoms and 4 electrons.
Molecule has automatically generated name H6_sto-3g_singlet_ring_0.7414
This molecule has 6 atoms and 6 electrons.
Molecule has automatically generated name H8_sto-3g_singlet_ring_0.7414
This molecule has 8 atoms and 8 electrons.
Molecule has automatically generated name H10_sto-3g_singlet_ring_0.7414
This molecule has 10 atoms and 10 electrons.
Molecule has automatically generated name H12_sto-3g_singlet_ring_0.7414
This molecule has 12 atoms and 12 electrons.
Molecule has automatically generated name H14_sto-3g_singlet_ring_0.7414
This molecule has 14 atoms and 14 electrons.
Molecule has automatically generated name H16_sto-3g_singlet_ring_0.7414
This molecule has 16 atoms and 16 electrons.
Molecule has automatically generated name H18_sto-3g_singlet_ring_0.7414
This molecule has 18 atoms and 18 electrons.


In [10]:
# Now let's use chemical_series.py to make some atoms.
print('\nIt can be hard to guess the right spin multiplicity of some atoms. Let FermiLib do it:')
atomic_symbols = ['C', 'Na', 'Ca', 'Te', 'Cu', 'Fe', 'Ag']
for symbol in atomic_symbols:
    atom = make_atom(symbol, basis, autosave)
    print('{} has multiplicity {} and would be named {}.'.format(
        symbol, atom.multiplicity, atom.name))


It can be hard to guess the right spin multiplicity of some atoms. Let FermiLib do it:
C has multiplicity 3 and would be named C1_sto-3g_triplet.
Na has multiplicity 2 and would be named Na1_sto-3g_doublet.
Ca has multiplicity 1 and would be named Ca1_sto-3g_singlet.
Te has multiplicity 3 and would be named Te1_sto-3g_triplet.
Cu has multiplicity 2 and would be named Cu1_sto-3g_doublet.
Fe has multiplicity 5 and would be named Fe1_sto-3g_quintet.
Ag has multiplicity 2 and would be named Ag1_sto-3g_doublet.


## Running Psi4 to populate MolecularData class

In [11]:
from run_psi4 import run_psi4

# Set molecule parameters.
basis = 'sto-3g'
multiplicity = 1
bond_length_interval = 0.5
n_points = 5

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1

# Generate molecule at different bond lengths.
for point in range(1, n_points + 1):
    bond_length = bond_length_interval * float(point)
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
    molecule = MolecularData(
        geometry, basis, multiplicity, autosave=False, description=str(bond_length))
    
    # Run Psi4.
    molecule = run_psi4(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

    # Print out some results of calculation.
    print('\nAt bond length of {} Bohr, molecular hydrogen has:'.format(bond_length))
    print('Hartree-Fock energy of {} Hartree.'.format(molecule.hf_energy))
    print('MP2 energy of {} Hartree.'.format(molecule.mp2_energy))
    print('FCI energy of {} Hartree.'.format(molecule.fci_energy))
    print('Nuclear repulsion energy between protons is {} Hartree.'.format(molecule.nuclear_repulsion))
    for orbital in range(molecule.n_orbitals):
        print('Spatial orbital {} has energy of {} Hartree.'.format(orbital, molecule.orbital_energies[orbital]))


At bond length of 0.5 Bohr, molecular hydrogen has:
Hartree-Fock energy of -1.04299627651 Hartree.
MP2 energy of -1.05148447259 Hartree.
FCI energy of -1.0551597965 Hartree.
Nuclear repulsion energy between protons is 1.05835441718 Hartree.
Spatial orbital 0 has energy of -0.690822327512 Hartree.
Spatial orbital 1 has energy of 0.988673669374 Hartree.

At bond length of 1.0 Bohr, molecular hydrogen has:
Hartree-Fock energy of -1.06610864808 Hartree.
MP2 energy of -1.08666270096 Hartree.
FCI energy of -1.1011503293 Hartree.
Nuclear repulsion energy between protons is 0.52917720859 Hartree.
Spatial orbital 0 has energy of -0.484441678962 Hartree.
Spatial orbital 1 has energy of 0.457501936164 Hartree.

At bond length of 1.5 Bohr, molecular hydrogen has:
Hartree-Fock energy of -0.910873552617 Hartree.
MP2 energy of -0.956287593623 Hartree.
FCI energy of -0.998149352414 Hartree.
Nuclear repulsion energy between protons is 0.352784805727 Hartree.
Spatial orbital 0 has energy of -0.35547748

## $\sum_{pq} h_{pq}\, a^\dagger_p a_q + \frac{1}{2} \sum_{pqrs} h_{pqrs} \, a^\dagger_p a^\dagger_q a_r a_s$ is a MolecularOperator

In [12]:
import scipy
import scipy.linalg
import numpy

# Set molecule parameters.
diatomic_bond_length = 4.
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
description = str(diatomic_bond_length)

# Set Hamiltonian parameters.
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData using Psi4.
molecule = MolecularData(geometry, basis, multiplicity,
                         description=description, autosave=False)
molecule = run_psi4(molecule, run_scf=True)

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    active_space_start, active_space_stop)
print('\nThe associated molecular Hamiltonian follows:\n{}\n'.format(molecular_hamiltonian))


The associated molecular Hamiltonian follows:
[] -7.00307139836

[0 0] -0.49176143122
[0 2] 0.127933719693
[1 1] -0.49176143122
[1 3] 0.127933719693
[2 0] 0.127933719693
[2 2] -0.389242464849
[3 1] 0.127933719693
[3 3] -0.389242464849

[0 1 1 0] 0.180809315862
[0 1 1 2] -0.0639668602239
[0 1 3 0] -0.0639668602239
[0 1 3 2] 0.0771665402713
[0 2 0 2] 0.0771665402713
[0 2 2 0] 0.144507326259
[0 3 1 0] -0.0639668602239
[0 3 1 2] 0.0771665402713
[0 3 3 0] 0.144507326259
[0 3 3 2] -0.0254096944453
[1 0 0 1] 0.180809315862
[1 0 0 3] -0.0639668602239
[1 0 2 1] -0.0639668602239
[1 0 2 3] 0.0771665402713
[1 2 0 1] -0.0639668602239
[1 2 0 3] 0.0771665402713
[1 2 2 1] 0.144507326259
[1 2 2 3] -0.0254096944453
[1 3 1 3] 0.0771665402713
[1 3 3 1] 0.144507326259
[2 0 0 2] 0.144507326259
[2 0 2 0] 0.0771665402713
[2 1 1 0] -0.0639668602239
[2 1 1 2] 0.144507326259
[2 1 3 0] 0.0771665402713
[2 1 3 2] -0.0254096944453
[2 3 1 0] 0.0771665402713
[2 3 1 2] -0.0254096944453
[2 3 3 0] -0.0254096944453
[2 3 

In [14]:
# Map operator to fermions and qubits.
fermion_hamiltonian = molecular_hamiltonian.get_fermion_operator()
qubit_hamiltonian = molecular_hamiltonian.jordan_wigner_transform()
print('The associated qubit Hamiltonian follows:\n{}'.format(qubit_hamiltonian))
sparse_hamiltonian = qubit_hamiltonian.get_sparse_operator()
energy, state = sparse_hamiltonian.get_ground_state()
print('Ground state energy before rotation is {} Hartree.'.format(energy))

# Randomly rotate.
n_orbitals = molecular_hamiltonian.n_qubits // 2
n_variables = n_orbitals * (n_orbitals - 1) / 2
random_angles = numpy.pi * (1. - 2. * numpy.random.rand(n_variables))
kappa = numpy.zeros((n_orbitals, n_orbitals))
index = 0
for p in range(n_orbitals):
    for q in range(p + 1, n_orbitals):
        kappa[p, q] = random_angles[index]
        kappa[q, p] = -numpy.conjugate(random_angles[index])
        index += 1

    # Build the unitary rotation matrix.
    difference_matrix = kappa + kappa.transpose()
    assert numpy.amax(numpy.absolute(difference_matrix)) < 1e-9
    rotation_matrix = scipy.linalg.expm(kappa)

    # Apply the unitary.
    molecular_hamiltonian.rotate_basis(rotation_matrix)
qubit_hamiltonian = molecular_hamiltonian.jordan_wigner_transform()
sparse_hamiltonian = qubit_hamiltonian.get_sparse_operator()
energy, state = sparse_hamiltonian.get_ground_state()
print('Ground state energy after rotation is {} Hartree.'.format(energy))

The associated qubit Hamiltonian follows:
(0.0192785825117+0j) X1 Z2 X3
(0.031983430112+0j) Z0 X1 Z2 X3
(0.0190962435726+0j) Z2
(0.031983430112+0j) X0 X2
(0.031983430112+0j) Z0 Y1 Z2 Y3
(0.0904046579309+0j) Z0 Z1
(0.0127048472226+0j) Y1 Y3
(0.0127048472226+0j) X0 Z1 X2 Z3
(-0.0385832701357+0j) X0 X1 Y2 Y3
(0.0495520015552+0j) Z0
(0.033670392994+0j) Z1 Z3
(0.0495520015552+0j) Z1
(0.0192785825117+0j) X0 Z1 X2
(0.0127048472226+0j) Y0 Z1 Y2 Z3
(0.0192785825117+0j) Y0 Z1 Y2
(0.033670392994+0j) Z0 Z2
(0.069600932728+0j) Z2 Z3
(0.0385832701357+0j) Y0 X1 X2 Y3
(0.0190962435726+0j) Z3
(0.0722536631297+0j) Z1 Z2
(0.0192785825117+0j) Y1 Z2 Y3
(0.031983430112+0j) Y0 Y2
(0.0722536631297+0j) Z0 Z3
(-0.0385832701357+0j) Y0 Y1 X2 X3
(-7.51222159152+0j) I
(0.0127048472226+0j) X1 X3
(0.0385832701357+0j) X0 Y1 Y2 X3

Ground state energy before rotation is -7.74939372245 Hartree.
Ground state energy after rotation is -7.74939372245 Hartree.
